In [1]:
'''INITIAL CURVE SETUP
======================
1) DATA
========
- Historical Swap Data (swap_data)
- Normal Volatility Data (vol_data)
- Historical Zero Coupon Curves (zeroCoupons)
- Historical Forward Curves (forwardCurves)

2) FUNCTIONS
=============
- Zero Coupon interpolator (zeroCouponInterpolator)
- Forward Interpolator (forwardRate)
- Forward Swap Rate Interpolator (forwardSwapRate)

3) PACKAGES
=============
Set1: math, numpy, pandas, itertools, matplotlib
Set2: scipy.stats, scipy.optimize, datetime, time, relativedelta
Set3: nelson_siegel_svensson
Set4: prettytable
'''

# Initialize Curve Setup
%run CurveSetup.ipynb

In [2]:
# Extra Packages
import re
import scipy.optimize as opt
from lmfit import minimize, Parameters, Parameter, report_fit
from itertools import compress

In [3]:
'''PRICERS
==============='''
' Black Pricer - Payer'
def blackPayer(N, S0, K, evalDate, maturityDate, tenorDate, sigma):
    eDate = datetime.strptime(evalDate, '%Y-%m-%d')
    maturity = datetime.strptime(maturityDate, '%Y-%m-%d')
    tenor = datetime.strptime(tenorDate, '%Y-%m-%d')
    
    # Find all payment dates for the swaption
    paymentDates = [datetime.strftime(maturity + relativedelta(years = i+1), '%Y-%m-%d') 
                    for i in range(tenor.year - maturity.year)]
    
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(zeroCouponVect(evalDate, paymentDates))
    
    # Define Black Parameters
    T = (maturity - eDate).days/365
    d1 = (log(S0/K) + 0.5*(pow(sigma,2)*T))/(sigma * sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    
    price = N*PVBP * (S0*stats.norm.cdf(d1) - K*stats.norm.cdf(d2))
    return(price)


' Chi Square Pricer - Payer'
def chiSquarePayer(N, S0, K, evalDate, maturityDate, tenorDate, sigma, beta, delta):
    eDate = datetime.strptime(evalDate, '%Y-%m-%d')
    maturity = datetime.strptime(maturityDate, '%Y-%m-%d')
    tenor = datetime.strptime(tenorDate, '%Y-%m-%d')
    
    # Find all payment dates for the swaption
    paymentDates = [datetime.strftime(maturity + relativedelta(years = i+1), '%Y-%m-%d') 
                    for i in range(tenor.year - maturity.year)]
    
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(zeroCouponVect(evalDate, paymentDates))
    
    # Parameters for input into Chi Square CDF
    T = (maturity - eDate).days/365
    d = (pow(K+delta,2 - 2*beta))/(pow(1 - beta,2)*pow(sigma,2)* T)
    b = 1/(1 - beta)
    f = (pow(S0+delta, 2 - 2*beta))/(pow(1 - beta, 2)*pow(sigma, 2)* T)

    # Calculate Price
    price = N * PVBP * ((S0+delta)*(1 - stats.ncx2.cdf(d, b+2, f)) - (K+delta)*(stats.ncx2.cdf(f, b, d)))
    return(price)

' Normal Pricer - Payer'
def normalPayer(N, S0, K, evalDate, maturityDate, tenorDate, sigma):
    eDate = datetime.strptime(evalDate, '%Y-%m-%d')
    maturity = datetime.strptime(maturityDate, '%Y-%m-%d')
    tenor = datetime.strptime(tenorDate, '%Y-%m-%d')
    
    # Find all payment dates for the swaption
    paymentDates = [datetime.strftime(maturity + relativedelta(years = i+1), '%Y-%m-%d') 
                    for i in range(tenor.year - maturity.year)]
    
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(zeroCouponVect(evalDate, paymentDates))
    
     # Define Black Parameters
    T = (maturity - eDate).days/365
    d1 = (S0 - K)/(sigma * sqrt(T))
    
    #Calculate Price
    price = N * PVBP * ((S0 - K)*stats.norm.cdf(d1) + (sigma * sqrt(T)* exp(-0.5 * pow(d1, 2)))/sqrt(2 * pi))
    return(price)

In [5]:
'''VECTORIZING ALL FUNCTIONS
======================================'''
zeroCouponVect = np.vectorize(zeroCouponInterpolator)
forwardRateVect = np.vectorize(forwardRate)
BlackVect = np.vectorize(blackPayer)
ChiSquareVect = np.vectorize(chiSquarePayer)
forwardSwapRateVect = np.vectorize(forwardSwapRate)
normalPayerVect = np.vectorize(normalPayer)

In [6]:
'''SELECT SWAPTIONS
====================================='''
testVolData = vol_data.loc['2019-12-31']

'''FIND STRIKES FOR EACH SWAPTION'''
tenorMaturities =  [(int(re.findall('[0-9]+',string)[0]),int(re.findall('[0-9]+',string)[1]) ) 
                    for string in list(vol_data.columns)]

maturities = [datetime.strftime(datetime.strptime('2019-12-31','%Y-%m-%d') + 
                                relativedelta(years = tenorMaturities[i][0]), '%Y-%m-%d')
              for i in range(len(tenorMaturities))]

tenor = [datetime.strftime( datetime.strptime('2019-12-31','%Y-%m-%d') + 
             relativedelta(years = tenorMaturities[i][0]+ tenorMaturities[i][1]), '%Y-%m-%d')
                 for i in range(len(tenorMaturities))]

#Obtain all strikes
strikes = forwardSwapRateVect('2019-12-31',maturities,tenor) 

# Calculate value of all swaptions
normalPrices = normalPayerVect(100, strikes, strikes, maturities, tenor,testVolData)

TypeError: Expected unicode, got numpy.str_

In [6]:
'''SIGMA ALPHA BETA FUNCTION
============================='''
def sigmaAlphaBeta(evalDate, expiryDate, tenorDate, sigma1, sigma2, alpha1, alpha2, rho, eta, delta):
    eDate = datetime.strptime(evalDate, '%Y-%m-%d')
    expiry = datetime.strptime(expiryDate, '%Y-%m-%d')
    tenor = datetime.strptime(tenorDate, '%Y-%m-%d')
    
    # Obtain payment schedule for swaption
    paymentDates = [datetime.strftime(expiry + relativedelta(years = i+1), '%Y-%m-%d') 
                    for i in range(tenor.year - expiry.year)]
    
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(zeroCouponVect(evalDate, paymentDates))
    
    # Calculate the forward swap rate
    swapForward = forwardSwapRate(evalDate, expiryDate, tenorDate)
    
    # Calculate the ZC bond value at T_alpha and T_beta
    zcAlpha = zeroCouponInterpolator(evalDate, expiryDate)
    zcBeta = zeroCouponInterpolator(evalDate, tenorDate)
    
    # Calculate the forward rate for each payment date
    forwards = np.power(forwardRateVect(evalDate, paymentDates) + delta , eta)
    
    # Calculate delta weights
    '''To be confirmed'''
    deltaWeights = [(swapForward/(1 + forwards[i]))* 
                    ((zcBeta/(zcAlpha-zcBeta)) + np.sum(zeroCouponVect(evalDate, paymentDates[i:]))/PVBP)
                    for i in range(len(paymentDates))]
    
    # Calculate Alpha Exponential 
    alpha1Exponential = np.exp(-alpha1 * np.arange(1, tenor.year - eDate.year + 1))
    alpha2Exponential = np.exp(-alpha2 * np.arange(1, tenor.year - eDate.year + 1))
    
    # Upsilon Terms (From volatility parametrization)
    upsilon1Terms = sqrt(1 - pow(rho, 2)) * sigma2 * alpha1Exponential 
    upsilon2Terms = sigma2 * alpha2Exponential + rho * sigma1 * alpha1Exponential
    
    # Combine vectors
    deltaWeightsCombination = list(itertools.combinations_with_replacement(deltaWeights, 2))
    forwardsCombination = list(itertools.combinations_with_replacement(forwards, 2))

    # Find the product of vectors
    deltaWeightsProduct = [np.product(elem) for elem in deltaWeightsCombination]
    forwardsProduct = [np.product(elem) for elem in forwardsCombination]
    
    # Define indices
    indices = list(itertools.combinations_with_replacement(np.arange(expiry.year - eDate.year, 
                                                        tenor.year - eDate.year), 2))

    #Integral of terms
    sigma1Integrals = [np.dot(upsilon1Terms[max(i, j) - min(i, j):max(i, j)], upsilon1Terms[:min(i, j)])
     if i != j 
     else np.dot(upsilon1Terms[:i], upsilon1Terms[:i]) for (i,j) in indices ]

    sigma2Integrals = [np.dot(upsilon2Terms[max(i, j) - min(i, j):max(i, j)], upsilon2Terms[:min(i, j)])
     if i != j 
     else np.dot(upsilon2Terms[:i], upsilon2Terms[:i]) for (i,j) in indices ]

    # Calculate different weights
    s1 = sum(np.multiply(np.multiply(deltaWeightsProduct, forwardsProduct), sigma1Integrals))
    s2 = sum(np.multiply(np.multiply(deltaWeightsProduct, forwardsProduct), sigma2Integrals))
    T = (tenor - eDate).days/365
    
    return((s2 + s1)*T/ pow(swapForward + delta, 2*eta))

In [7]:
''' Chi Square Pricer - Payer
=============================='''
def calibrationChiSquarePayer(N, S0, K, evalDate, expiryDate, tenorDate, 
                   sigma1, sigma2, alpha1, alpha2, rho, eta, delta):
    eDate = datetime.strptime(evalDate, '%Y-%m-%d')
    maturity = datetime.strptime(expiryDate, '%Y-%m-%d')
    tenor = datetime.strptime(tenorDate, '%Y-%m-%d')
    
    # Find all payment dates for the swaption
    paymentDates = [datetime.strftime(maturity + relativedelta(years = i+1), '%Y-%m-%d') 
                    for i in range(tenor.year - maturity.year)]
    
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(zeroCouponVect(evalDate, paymentDates))
    
    # Parameters for input into Chi Square CDF
    sigma =  sigmaAlphaBeta(evalDate, expiryDate, tenorDate, sigma1, sigma2, alpha1, alpha2, rho, eta, delta)
    T = (maturity - eDate).days/365
    d = (pow(K+delta,2 - 2*eta))/(pow(1 - eta,2)*pow(sigma,2)* T)
    b = 1/(1 - eta)
    f = (pow(S0+delta, 2 - 2*eta))/(pow(1 - eta, 2)*pow(sigma, 2)* T)

    # Calculate Price
    price = N * PVBP * ((S0+delta)*(1 - stats.ncx2.cdf(d, b+2, f)) - (K+delta)*(stats.ncx2.cdf(f, b, d)))
    return(price)

# Vectorize Function
calibrationChiSquarePayerVect = np.vectorize(calibrationChiSquarePayer)

In [8]:
'''CALIBRATE INDIVIDUAL PARAMETERS FROM PRICES
==============================================='''
# Select swaptions between 5X5 and 10X10
weights = np.array([((tenorMaturity[0] >= 5) & (tenorMaturity[0] <= 10 ) & 
                        (tenorMaturity[1] >= 5) & (tenorMaturity[1] <= 10 )) 
                    for tenorMaturity in tenorMaturities])

strikesCalib = np.array(list(compress(strikes, weights)))
maturitiesCalib = list(compress(maturities, weights))
tenorCalib = list(compress(tenor, weights))
normalPricesCalib = list(compress(normalPrices, weights))

objectiveFunction = lambda test:  (calibrationChiSquarePayerVect(100, strikesCalib, 
                    strikesCalib, 
                    '2019-12-31', 
                    maturitiesCalib, 
                    tenorCalib, 
                    test['sigma1'].value, 
                    test['sigma2'].value, 
                    test['alpha1'].value, 
                    test['alpha2'].value,
                    test['rho'].value,
                    test['eta'].value , 0.05) - normalPricesCalib)

In [ ]:
# tbl = PrettyTable()
# header = ['Maturity', 'Tenor', 'Strike', 'Price']
# tbl.add_column(header[0], maturitiesCalib)
# tbl.add_column(header[1], tenorCalib)
# tbl.add_column(header[2], strikesCalib)
# tbl.add_column(header[3], normalPricesCalib)
# print(tbl)

In [9]:
# Set parameters and bounds
params = Parameters()
params.add('sigma1', value = 1,  min = 0, max = 1)
params.add('sigma2',value = 1,  min = 0, max = 1)
params.add('alpha1',value = 1,  min = 0, max = 1)
params.add('alpha2',value = 1,  min = 0, max = 1)
params.add('rho', value = 0, min = 0, max = 1)
params.add('eta', value = 0.5, min = 0, max = 1)

startTime = time()
result = minimize(objectiveFunction, 
                  params, 
                  method = 'least_sq')
endTime = time()
(endTime - startTime)/60

17.478814129034678

In [10]:
'''Print Calibration Results'''
headers = ["Iterations", "Sigma1", "Sigma2", "Alpha1", "Alpha2", "Rho", "Eta"]
t2 = PrettyTable()
t2.add_column(headers[0], [int(result.nfev)])
t2.add_column(headers[1], [round(result.params['sigma1'].value, 5)])
t2.add_column(headers[2], [round(result.params['sigma2'].value, 5)])
t2.add_column(headers[3], [round(result.params['alpha1'].value, 5)])
t2.add_column(headers[4], [round(result.params['alpha2'].value, 5)])
t2.add_column(headers[5], [round(result.params['rho'].value, 5)])
t2.add_column(headers[6], [round(result.params['eta'].value, 5)])
print(t2)

+------------+---------+---------+---------+---------+---------+---------+
| Iterations |  Sigma1 |  Sigma2 |  Alpha1 |  Alpha2 |   Rho   |   Eta   |
+------------+---------+---------+---------+---------+---------+---------+
|     83     | 0.99439 | 0.02878 | 0.16294 | 0.16284 | 0.00011 | 0.09788 |
+------------+---------+---------+---------+---------+---------+---------+


In [11]:
headers3 = ['Maturity', 'Tenor', 'Error(%)']
t3 = PrettyTable()
t3.add_column(headers3[0], maturitiesCalib)
t3.add_column(headers3[1], tenorCalib)
t3.add_column(headers3[2],list(np.round((objectiveFunction(result.params)/normalPricesCalib)*100, 2)))
print(t3)

+------------+------------+----------+
|  Maturity  |   Tenor    | Error(%) |
+------------+------------+----------+
| 2024-12-31 | 2034-12-31 |  -32.05  |
| 2024-12-31 | 2029-12-31 |  -18.87  |
| 2024-12-31 | 2030-12-31 |  -23.47  |
| 2024-12-31 | 2031-12-31 |  -26.73  |
| 2024-12-31 | 2032-12-31 |  -29.63  |
| 2024-12-31 | 2033-12-31 |  -31.28  |
| 2025-12-31 | 2035-12-31 |  -27.35  |
| 2025-12-31 | 2030-12-31 |  -17.64  |
| 2025-12-31 | 2031-12-31 |  -21.27  |
| 2025-12-31 | 2032-12-31 |  -24.48  |
| 2025-12-31 | 2033-12-31 |  -26.24  |
| 2025-12-31 | 2034-12-31 |  -27.24  |
| 2026-12-31 | 2036-12-31 |  -22.14  |
| 2026-12-31 | 2031-12-31 |  -14.99  |
| 2026-12-31 | 2032-12-31 |  -18.81  |
| 2026-12-31 | 2033-12-31 |  -20.82  |
| 2026-12-31 | 2034-12-31 |  -21.93  |
| 2026-12-31 | 2035-12-31 |  -22.22  |
| 2027-12-31 | 2037-12-31 |  -16.2   |
| 2027-12-31 | 2032-12-31 |  -11.92  |
| 2027-12-31 | 2033-12-31 |  -14.5   |
| 2027-12-31 | 2034-12-31 |  -15.88  |
| 2027-12-31 | 2035-12-31

In [ ]:
startTime = time()
resultAmpgo = minimize(objectiveFunction, 
                  params, 
                  method = 'ampgo')
endTime = time()
(endTime - startTime)/60

In [ ]:
startTime = time()
resultAmpgo = minimize(objectiveFunction, 
                  params, 
                  method = 'shgo')
endTime = time()
(endTime - startTime)/60